In [3]:
from Player import Player
from Trainer import Trainer
from TetrisModel import TetrisModel
from Pretrainer import Pretrainer
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import time

In [4]:
piece_dim = 8
key_dim = 12
depth = 32
gamma = 0.99
lam = 0.95

In [5]:
# Use lambda instead of gamma to immitate shape of gae without value predictions
pretrainer = Pretrainer(gamma=lam)

In [6]:
players_data = pretrainer._load_data()

In [7]:
pretrainer._load_dset(players_data)

1.00

In [8]:
max_len = pretrainer._max_len

In [9]:
gt_dset = pretrainer._cache_dset()

Done Caching


In [10]:
agent = TetrisModel(piece_dim=piece_dim,
                    key_dim=key_dim,
                    depth=depth,
                    num_heads=4,
                    num_layers=4,
                    max_length=max_len)

In [11]:
logits, values = agent((tf.random.uniform((1, 28, 10, 1)),
                        tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                        tf.random.uniform((1, max_len-1), minval=0, maxval=key_dim, dtype=tf.int32)))
agent.summary(), tf.shape(logits), tf.shape(values)

Model: "tetris_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (1, 70, 32)               18816     
                                                                 
 seq_embedding (SeqEmbedding  multiple                 256       
 )                                                               
                                                                 
 seq_embedding_1 (SeqEmbeddi  multiple                 384       
 ng)                                                             
                                                                 
 pdec_0 (DecoderLayer)       multiple                  37984     
                                                                 
 pdec_1 (DecoderLayer)       multiple                  37984     
                                                                 
 pdec_2 (DecoderLayer)       multiple                 

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 21, 12])>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 21,  1])>)

In [12]:
agent_optimizer = keras.optimizers.Adam()
agent.compile(optimizer=agent_optimizer)

In [13]:
epochs = 20

In [ ]:
pretrainer.train(agent, gt_dset, epochs)


6050	|	Loss: 51.40	|	Accuracy: 0.65	|	Step Time:  56ms	|	
6050	|	Loss: 49.13	|	Accuracy: 0.69	|	Step Time:  65ms	|	
6050	|	Loss: 42.50	|	Accuracy: 0.67	|	Step Time:  52ms	|	
6050	|	Loss: 40.86	|	Accuracy: 0.69	|	Step Time:  49ms	|	
6050	|	Loss: 35.22	|	Accuracy: 0.73	|	Step Time:  48ms	|	
6050	|	Loss: 42.74	|	Accuracy: 0.71	|	Step Time:  59ms	|	
4220	|	Loss: 19.21	|	Accuracy: 0.79	|	Step Time:  55ms	|	

In [29]:
plt.plot(losses)
plt.plot(accs)

In [18]:
agent.load_weights('agent_weights/agent')

In [24]:
agent.load_weights('agent_weights/agent')

In [30]:
if 'y' in input('YOU SURE?????'):
    agent.save_weights('agent_weights/agent')

YOU SURE????? y


In [17]:
%matplotlib qt

In [20]:
actor_optimizer = keras.optimizers.Adam()
critic_optimizer = keras.optimizers.Adam()

In [21]:
trainer = Trainer(model=agent,
                  optimizers=(actor_optimizer, critic_optimizer),
                  seq_len=max_len,
                  gamma=gamma,
                  lam=lam,
                  max_episode_steps=500,
                  buffer_cap=10000)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michaelsherrick. Use `wandb login --relogin` to force relogin


In [22]:
trainer.fill_replay_buffer()

Done filling replay buffer


In [23]:
trainer.train(gens=1000, train_steps=10, training_actor=True)

Current Gen: 1	|	Avg Reward: 0.1	|	Total Reward: 126.3	|
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Current Gen: 2	|	Avg Reward: 0.1	|	Total Reward: 121.4	|
Current Gen: 3	|	Avg Reward: 0.1	|	Total Reward: 122.4	|
Current Gen: 4	|	Avg Reward: 0.1	|	Total Reward: 133.9	|
Current Gen: 5	|	Avg Reward: 0.1	|	Total Reward: 90.8	|
Current Gen: 6	|	Avg Reward: 0.0	|	Total Reward: 11.1	|
Current Gen: 7	|	Avg Reward: 0.1	|	Total Reward: 121.9	|
Current Gen: 8	|	Avg Reward: 0.1	|	Total Reward: 134.5	|
Current Gen: 9	|	Avg Reward: 0.1	|	Total Reward: 93.2	|
Current Gen: 10	|	Avg Reward: 0.1	|	Total Reward: 142.7	|
Current Gen: 11	|	Avg Reward: 0.1	|	Total Reward: 106.4	|
Current Gen: 12	|	Avg Reward: 0.1	|	Total Reward: 140.5	|
Current Gen: 13	|	Avg Reward: 0.1	|	Total Reward: 74.4	|
Current Gen: 14	|	Avg Reward: 0.1	|	Total Reward: 146.2	|
Current Gen: 15	|	Avg Reward: 0.1	|	Total Reward: 142.7	|
Current Gen: 16	|	Avg Reward: 0.1	|	Total Reward: 157